## Chroma DB 테스트

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_and_store_chromadb(csv_file_path, chroma_path):
    # ChromaDB 초기화
    client = chromadb.PersistentClient(path=chroma_path)
    collection = client.get_or_create_collection(name="wanted_jobs")

    # 임베딩 모델 로딩
    embedding_model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")

    # Text Splitter 설정
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)

    # Selenium 환경 설정
    CHROMEDRIVER_PATH = r"./data/chromedriver.exe"
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    # CSV 파일 읽기
    df = pd.read_csv(csv_file_path)

    try:
        for idx, row in df.iterrows():
            job_url = row["URL"]
            print(f"\n🔍 공고 처리 중 ({idx+1}/{len(df)}): {job_url}")

            driver.get(job_url)
            time.sleep(2)

            wait = WebDriverWait(driver, 10)

            # 포지션명 수집
            try:
                position_name = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
            except:
                position_name = "N/A"

            # 경력 수집
            try:
                exp_elements = driver.find_elements(By.CSS_SELECTOR, "span.JobHeader_JobHeader__Tools__Company__Info__yT4OD")
                experience = exp_elements[1].text.strip() if len(exp_elements) > 1 else "N/A"
            except:
                experience = "N/A"

            # 상세 정보 버튼 클릭
            try:
                more_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(),'상세 정보 더 보기')]]")))
                driver.execute_script("arguments[0].click();", more_btn)
                time.sleep(2)
            except:
                pass

            # 섹션 구분하여 크롤링 (필수 섹션만)
            job_description_list = []
            current_section = None

            try:
                desc_container = driver.find_element(By.CSS_SELECTOR, "article.JobDescription_JobDescription__dq8G5")
                elements = desc_container.find_elements(By.XPATH, ".//*")

                for el in elements:
                    tag_name = el.tag_name.lower()
                    text = el.text.strip()

                    if tag_name == "h2":
                        job_description_list.append(f"[{text}]")

                    elif tag_name == "h3":
                        if "주요업무" in text:
                            current_section = "MainTask"
                            job_description_list.append(f"\n{text}")
                        elif "자격요건" in text:
                            current_section = "Qualification"
                            job_description_list.append(f"\n{text}")
                        elif "우대사항" in text:
                            current_section = "Preferred"
                            job_description_list.append(f"\n{text}")
                        else:
                            current_section = None  # 불필요 섹션 제외

                    elif tag_name == "span" and current_section:
                        job_description_list.append(text)

            except Exception as e:
                print(f" 크롤링 중 오류 발생: {e}")
                continue

            # 분할 및 임베딩 처리
            full_text = "\n".join(job_description_list).strip()
            if not full_text:
                continue

            split_chunks = text_splitter.split_text(full_text)
            embeddings = embedding_model.encode(split_chunks).tolist()

            # ✅ 최종적으로 올바른 DB 저장 방식
            for i, chunk in enumerate(split_chunks):
                collection.add(
                    ids=[f"{idx}-{i}"],
                    documents=[chunk],  # ✅ 문자열 필수
                    embeddings=[embeddings[i]],
                    metadatas=[{
                        "PositionName": position_name,
                        "Experience": experience
                    }]
                )

            print(f"✅ 저장 완료: {position_name} ({len(split_chunks)}개 문장)")

    except Exception as e:
        print(f" 전반적인 오류 발생: {e}")

    finally:
        driver.quit()
        print("\n 모든 작업이 완료되었습니다. ChromaDB 저장 성공.")

# 최종 코드 실행
input_csv = "./wanted_merged(1).csv"
chroma_db_path = "./data/chroma_db"

scrape_and_store_chromadb(input_csv, chroma_db_path)


c:\Users\user\miniconda3\envs\job311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔍 공고 처리 중 (1/3): https://www.wanted.co.kr/wd/270338
✅ 저장 완료: NLP AI 엔지니어 (4개 문장)

🔍 공고 처리 중 (2/3): https://www.wanted.co.kr/wd/263703
✅ 저장 완료: AI 사업계획서/제안서 작성 담당자 (2개 문장)

🔍 공고 처리 중 (3/3): https://www.wanted.co.kr/wd/268265
✅ 저장 완료: AI솔루션 영업직 (4개 문장)

🎯 모든 작업이 완료되었습니다. ChromaDB 저장 성공.


## RAG 테스트

In [ ]:
import chromadb
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import os

# API 키 설정 
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# ChromaDB 경로 설정
chroma_db_path = "./data/chroma_db"

# Embedding 모델 설정 (ChromaDB 저장 시 사용한 것과 동일한 모델)
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct"
)

# ChromaDB에서 벡터DB 로드 (LangChain 인터페이스 사용)
vectorstore = Chroma(
    persist_directory=chroma_db_path,
    embedding_function=embedding_model
)

# Retriever 설정 (유사한 문서 2개 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k":2})

# LLM 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 사용자 프로필 입력 예시 (실제 서비스에서는 사용자 입력을 활용)
user_profile = """
저는 신입입니다. LLM을 이용하여 가사 생성, 일기 생성의 프로젝트 경험이 있고, pandas, git, python, transformers,pytorch 등 사용 경험이 있습니다.
나한테 알맞는 직무가 모가 있나요?
"""

# Retriever를 통해 문서 검색
retrieved_docs = retriever.get_relevant_documents(user_profile)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# RAG를 위한 명확한 프롬프트 설정
template = """
당신은 채용 전문가입니다. 주어진 내용을 참고하여 구직자에게 가장 적합한 포지션을 추천하세요.

[내용]:
{context}

[구직자 프로필]:
{user_profile}

추천 가이드라인:
- 구직자에게 가장 잘 맞는 포지션을 최대 2개 추천합니다.
- 각 포지션의 포지션명, 요구 경력, 주요 업무, 자격요건, 우대사항을 명확히 제시합니다.
- 각 추천에 대해 구직자의 프로필과 어떤 점이 일치하는지 명확히 제시합니다.
- 추천의 이유도 제시합니다.
                                          

추천 결과:
"""

prompt = ChatPromptTemplate.from_template(template)

# 프롬프트 완성 후 LLM 전달
final_prompt = prompt.format_messages(context=context, user_profile=user_profile)

# LLM 호출 
response = llm.invoke(final_prompt)

# 최종 결과 출력
print(" RAG 구직자 맞춤형 추천 결과 ")
print(response.content)


🔥 RAG 구직자 맞춤형 추천 결과 🔥
구직자님의 프로필을 바탕으로 다음 두 가지 포지션을 추천드립니다.

### 1. 포지션명: 데이터 분석가 (Data Analyst)

- **요구 경력**: 신입
- **주요 업무**:
  - 데이터 수집 및 정제
  - 데이터 분석 및 시각화
  - 비즈니스 인사이트 도출
- **자격요건**:
  - Python 및 Pandas 사용 경험
  - 데이터 분석 및 시각화 도구에 대한 이해
  - 기본적인 통계 지식
- **우대사항**:
  - Git을 통한 버전 관리 경험
  - 머신러닝 또는 데이터 마이닝 관련 프로젝트 경험

**추천 이유**: 구직자님은 Python과 Pandas를 사용한 경험이 있으며, 데이터 처리 및 분석에 대한 기초적인 이해가 있을 것으로 보입니다. 데이터 분석가는 데이터 기반의 의사결정을 지원하는 역할로, 구직자님의 기술적 배경이 잘 맞아떨어집니다. 또한, 신입 포지션이기 때문에 경력에 대한 부담이 적습니다.

---

### 2. 포지션명: 머신러닝 엔지니어 (Machine Learning Engineer)

- **요구 경력**: 신입
- **주요 업무**:
  - 머신러닝 모델 개발 및 배포
  - 데이터 전처리 및 모델 학습
  - 성능 평가 및 최적화
- **자격요건**:
  - Python 및 PyTorch 사용 경험
  - 머신러닝 및 딥러닝 기본 지식
  - LLM(대형 언어 모델) 관련 프로젝트 경험
- **우대사항**:
  - Transformers 라이브러리 사용 경험
  - Git을 통한 협업 경험

**추천 이유**: 구직자님은 LLM을 이용한 프로젝트 경험이 있으며, PyTorch와 Transformers에 대한 사용 경험도 가지고 있습니다. 머신러닝 엔지니어는 이러한 기술을 활용하여 모델을 개발하고 최적화하는 역할을 수행하므로, 구직자님의 경험이 매우 적합합니다. 또한, 신입 포지션으로 경력에 대한 부담이 적어 도전하기 좋은 기회입니다.

이 두 가지 포지션은 구직자